# LangChain for LLM Application Development from DeepLearning.AI
## Lesson 2: Memory

In [6]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

load_dotenv('OPENAI_API_KEY.env')  # Load the specified .env file
print(os.environ)  # Print all environment variables
print(os.environ.get('OPENAI_API_KEY'))  # Check specifically for your key

openai.api_key = os.environ['OPENAI_API_KEY']  # Access your API key



environ({'TMPDIR': '/var/folders/9k/956xnzkj6gj30_bckgwcyl3m0000gn/T/', '__CFBundleIdentifier': 'com.apple.Terminal', 'XPC_FLAGS': '0x0', 'LaunchInstanceID': '7DB4FF6D-56CE-42BE-86CD-6F2D0ABEAB5B', 'TERM': 'xterm-color', 'SSH_AUTH_SOCK': '/private/tmp/com.apple.launchd.WPQ4qewqks/Listeners', 'SECURITYSESSIONID': '18e05', 'XPC_SERVICE_NAME': '0', 'TERM_PROGRAM': 'Apple_Terminal', 'TERM_PROGRAM_VERSION': '454.1', 'TERM_SESSION_ID': 'ADB6D160-EB67-4962-A917-D59C584AE1F4', 'SHELL': '/bin/zsh', 'HOME': '/Users/hclee', 'LOGNAME': 'hclee', 'USER': 'hclee', 'PATH': '/opt/anaconda3/bin:/opt/anaconda3/condabin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/appleinternal/bin', 'SHLVL': '1', 'PWD': '/Users/hclee', 'OLDPWD': '/Users/hclee', 'CONDA_EXE': '/opt/anaconda3

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
import warnings
warnings.filterwarnings('ignore')

In [8]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

### Conversation Buffer Memory

In [9]:
# Define model using ConversationBufferMemory
llm = ChatOpenAI(temperature=0.0, model=llm_model)
# creates memory buffer to record history of chat
memory = ConversationBufferMemory()
# creates langchain
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [10]:
conversation.predict(input="Hi, my name is Howard")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Howard
AI:

> Finished chain.


"Hello Howard! It's nice to meet you. How can I assist you today?"

In [11]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Howard
AI: Hello Howard! It's nice to meet you. How can I assist you today?
Human: What is my name?
AI:

> Finished chain.


'Your name is Howard.'

In [12]:
conversation.predict(input="What is your name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Howard
AI: Hello Howard! It's nice to meet you. How can I assist you today?
Human: What is my name?
AI: Your name is Howard.
Human: What is your name?
AI:

> Finished chain.


"I am an AI assistant, so I don't have a personal name like humans do. You can just call me AI or assistant."

In [14]:
# stores entire conversation in memory buffer
print(memory.buffer)

Human: Hi, my name is Howard
AI: Hello Howard! It's nice to meet you. How can I assist you today?
Human: What is my name?
AI: Your name is Howard.
Human: What is your name?
AI: I am an AI assistant, so I don't have a personal name like humans do. You can just call me AI or assistant.


In [15]:
# you can also retrieve using function load_memory_variables in the format of dict {}, key 'history'
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Howard\nAI: Hello Howard! It's nice to meet you. How can I assist you today?\nHuman: What is my name?\nAI: Your name is Howard.\nHuman: What is your name?\nAI: I am an AI assistant, so I don't have a personal name like humans do. You can just call me AI or assistant."}

In [16]:
# can also use Class ConversationBufferMemory to store only the context
memory = ConversationBufferMemory()

In [17]:
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

In [18]:
print(memory.buffer)

Human: Hi
AI: What's up


In [19]:
# adding to history using function save_context
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

In [20]:
print(memory.buffer)

Human: Hi
AI: What's up
Human: Not much, just hanging
AI: Cool


### Conversation Buffer Window Memory

In [21]:
from langchain.memory import ConversationBufferWindowMemory

In [26]:
# k=1 specifies memory to store only 1 conversational exchange. Last In First Out basis. Helps to reduce the size \n
# of context window
memory = ConversationBufferWindowMemory(k=1)               

In [23]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})


In [24]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [25]:
# Define model using ConversationBufferWindowMemory

llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

In [27]:
# Object memory now has no memory of conversational exchanges greater than 1
conversation.predict(input="What is my name?")

"I'm sorry, I do not have access to personal information such as your name."

### Conversation Token Buffer Memory
- limiting costs using cap on tokens, rather than conversational exchanges

In [28]:
!pip install tiktoken

     |████████████████████████████████| 961 kB 745 kB/s eta 0:00:01
     |████████████████████████████████| 287 kB 9.5 MB/s eta 0:00:01
  Using cached requests-2.32.3-py3-none-any.whl (64 kB)
     |████████████████████████████████| 124 kB 21.2 MB/s eta 0:00:01
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
  Attempting uninstall: regex
    Found existing installation: regex 2021.4.4
    Uninstalling regex-2021.4.4:
      Successfully uninstalled regex-2021.4.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
anaconda-project 0.9.1 requires ruamel-yaml, which is not installed.


In [29]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0, model=llm_model)

In [30]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [31]:
# 50 tokens cuts off the first input, again Last in First out
# Different LLMs use different methods for counting tokens
memory.load_memory_variables({})

{'history': 'AI: Amazing!\nHuman: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

### Produce summaries using ConversationSummaryBufferMemory

In [32]:
from langchain.memory import ConversationSummaryBufferMemory

In [33]:
# create a long string, then apply SummaryBufferMemory with token limit, note syntax "output":f"{object}"}
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [34]:
memory.load_memory_variables({})

{'history': "System: The human and AI exchange greetings and discuss the schedule for the day. The AI informs the human of a morning meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of being prepared for the day's events."}

In [35]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [36]:
conversation.predict(input="Where would be a good place to hold the lunch meeting?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI exchange greetings and discuss the schedule for the day. The AI informs the human of a morning meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of being prepared for the day's events.
Human: Where would be a good place to hold the lunch meeting?
AI:

> Finished chain.


"There are several options for a good place to hold the lunch meeting with the customer interested in AI. One option could be a trendy cafe downtown that has a quiet atmosphere conducive to discussions. Another option could be a restaurant with a private dining area where we can have more privacy. I can also look up some highly rated restaurants in the area if you'd like."

In [37]:
memory.load_memory_variables({})

{'history': "System: The human and AI exchange greetings and discuss the schedule for the day. The AI informs the human of a morning meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of being prepared for the day's events.\nHuman: Where would be a good place to hold the lunch meeting?\nAI: There are several options for a good place to hold the lunch meeting with the customer interested in AI. One option could be a trendy cafe downtown that has a quiet atmosphere conducive to discussions. Another option could be a restaurant with a private dining area where we can have more privacy. I can also look up some highly rated restaurants in the area if you'd like."}